In [68]:
from pythonosc.udp_client import SimpleUDPClient

import pandas as pd
import numpy as np
from random import randint
from time import sleep, time
from mido import MidiFile

In [2]:
# read the osc data and check min/max
df_score = pd.read_csv('data\osc_data.csv')
df_score[[col for col in df_score.columns if '_ts' in col]].describe()

,low_pitch_val_ts12,low_pitch_val_ts16,high_pitch_val_ts12,high_pitch_val_ts16,synth_val_ts
count,56.000000,56.000000,56.000000,56.000000,56.000000
mean,8.000000,9.821429,6.803571,8.178571,4.642857
std,2.215647,3.379656,2.726434,3.968545,1.518714
min,5.000000,5.000000,4.000000,4.000000,1.000000
25%,6.000000,7.000000,4.000000,4.000000,4.000000
50%,8.000000,10.000000,6.000000,7.500000,5.000000
75%,9.000000,12.000000,9.000000,11.000000,6.000000
max,12.000000,16.000000,13.000000,17.000000,7.000000


In [8]:
# make a variable to send OSC messages
osc_power = SimpleUDPClient("127.0.0.1", 8000)
osc_messenger = SimpleUDPClient("127.0.0.1", 8001)

# update the changes with change in cycles
cycle_change = [0,8,30,36,44]
metro_change = [140,100,110,160,120]   
change = 0
bpm = 120


# loop for all the cycles
for i in range(df_score.shape[0]):

    # get current cycle info
    info = df_score.iloc[i]

    #check if metro needs to be updated
    if (change<len(cycle_change) and i == cycle_change[change]):
        bpm = metro_change[change]
        change = change + 1
        osc_power.send_message('/metro', bpm)
    
    # send needed info to the Pd patch
    osc_messenger.send_message('/drum_hi_ts', info['high_pitch_val_ts16'])
    osc_messenger.send_message('/drum_low_ts', info['low_pitch_val_ts16'])
    osc_messenger.send_message('/groove_seed', change)

    # reset the bpm button
    osc_messenger.send_message('/set_bpm', 1)

    # start the BPM if its first cycle
    if i == 0:
        osc_power.send_message('/power',1)

    # add a delay according to the metro for updated values
    sleep(int(8*60/bpm))

# end the BPM on pureData
osc_power.send_message('/power',0)

In [4]:
import time

start = (time.time())

# update the changes with change in cycles
cycle_change = [0,8,30,36,44]
metro_change = [140,100,110,160,120]   
change = 0
bpm = 120


# loop for all the cycles
for i in range(df_score.shape[0]):

    #check if metro needs to be updated
    if (change<len(cycle_change) and i == cycle_change[change]):
        print(change, time.time()-start)
        bpm = metro_change[change]
        change = change + 1

    # add a delay according to the metro for updated values
    sleep(int(8*60/bpm))

0 0.0004963874816894531
1 24.0046443939209
2 112.0149040222168
3 136.0243308544159
4 160.10480499267578


In [93]:
files = []

# files.append('PolyGlucose_04_01.mid')
# files.append('PolyGlucose_04_02.mid')
# files.append('PolyGlucose_04_03.mid')
# files.append('PolyGlucose_04_04_01.mid')
files.append('PolyGlucose_04_04_02.mid')
files.append('PolyGlucose_04_05.mid')

# make a variable to send OSC messages
osc_messenger = SimpleUDPClient("127.0.0.1", 8001)

bass_no = [60,61,62,63,64,65,66,67]
snare_no = [52,53,54,55]
hihat_no = [56,57,58,59]

sat_range = [0,2]
con_range = [0.75,2]
amt_range = [0,1]
gain_range = [0,1]

sat = 1
con = 1
change = 0.02

amt = 127
gain = 127

rand_state = -1
rand_time = 2

start = time()

# let the midi file play itself with the right time
for i, file in enumerate(files):
    # read the needed MIDI file
    name = 'GlucosePiece Project\\MIDI samples\\PolyGlucose 04 parts\\used\\' + file
    print(name)
    mid = MidiFile(name)

    print(i, time()-start, 'working on ' + file)

    for msg in mid.play():
        if not msg.is_meta:
            if msg.type == 'note_on':
        
                if msg.note in bass_no:
                    osc_messenger.send_message('/drum_bass', msg.note - 60 + 1)
                    
                    sat -= change
                    amt += randint(0,rand_time)*rand_state
                elif msg.note in hihat_no:
                    osc_messenger.send_message('/drum_hihat', msg.note - 56 + 1)
                    
                    sat += change
                    con += change
                    amt -= randint(0,rand_time)
                    gain -= randint(0,rand_time)

                elif msg.note in snare_no:
                    osc_messenger.send_message('/drum_snare', msg.note - 52 + 1)
                    
                    con -= change
                    gain += randint(0,rand_time) * rand_state

            elif msg.type == 'note_off':
                send = 0

                if msg.note in bass_no:
                    osc_messenger.send_message('/drum_bass', send)
                elif msg.note in hihat_no:
                    osc_messenger.send_message('/drum_hihat', send)
                elif msg.note in snare_no:
                    osc_messenger.send_message('/drum_snare', send)

            osc_messenger.send_message('/sat', sat)
            osc_messenger.send_message('/con', con)
            osc_messenger.send_message('/amt', amt)
            osc_messenger.send_message('/gain', gain)

            if amt < 30 and gain < 30:
                rand_state = 1
                rand_time = 1

GlucosePiece Project\MIDI samples\PolyGlucose 04 parts\used\PolyGlucose_04_04_02.mid
0 0.0029757022857666016 working on PolyGlucose_04_04_02.mid
GlucosePiece Project\MIDI samples\PolyGlucose 04 parts\used\PolyGlucose_04_05.mid
1 14.863717317581177 working on PolyGlucose_04_05.mid
